In [ ]:
!pip install loguru transformers

In [4]:
!./train.sh

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertGenerationEncoder: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'bert.embeddings.token_type_embeddings.weight']
- This IS expected if you are initializing BertGenerationEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertGenerationEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights 

In [9]:
import sagemaker
from sagemaker.pytorch import PyTorch

role = sagemaker.get_execution_role()
output_path = f's3://chegg-ds-data/oboiko/wdm-output'

pytorch_estimator = PyTorch(entry_point='train.sh',
                            base_job_name='wdm-mp',
                            role=role,
                            train_instance_count=1,
                            train_instance_type='ml.p2.8xlarge',  # GPU instance
                            train_volume_size=50,
                            train_max_run=86400,  # 24 hours
                            hyperparameters={
                              'model_type': 'bert-base-uncased',
                              'data_loc': '/opt/data',
                              'batch': 48,
                              'epochs': 10,
                              'lr': 1e-5,
                              'train_remotely': 0,
                              'notebook_name': 'main_bert_distributed',
                              'output_dir': '/opt/ml/model'  
                            },
                            framework_version='1.6.0',
                            py_version='py3',
                            source_dir='.',  # This entire folder will be transferred to training instance
                            debugger_hook_config=False,
                            output_path=output_path,  # Model files will be uploaded here
                            image_name='954558792927.dkr.ecr.us-west-2.amazonaws.com/sagemaker/wdm:latest',
                            metric_definitions=[
                                {'Name': 'train:error', 'Regex': 'train_error=(.*?);'},
                                {'Name': 'validation:error', 'Regex': 'valid_error=(.*?);'},
                                {'Name': 'batch:error', 'Regex': 'batch_error=(.*?);'}
                            ]
                 )

pytorch_estimator.fit('s3://chegg-ds-data/oboiko/wdm/dummy.txt', wait=False)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
